<a href="https://colab.research.google.com/github/g-roger/natural-language-process/blob/main/Aula_4_6_IA_PLN_Exercicio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Exercício 2**

Dado o dataset de produtos [1], descubra:
- Treine deferentes modelos de classificação do pacote scikit-learn para classificar os produtos em suas categorias.
- Experimente a lib SpaCy para remover as stop words e reduzir as palavras ao seu lema. Veja como essas alterações impactam o desempenho do classificador.
- Compare a performance entre eles usando a métrica de acurácia.
- Use randon_state igual a 42 para permitir a comparação com seus colegas, separando 30% para teste.


[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv

Carregando os dados

In [ ]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')

In [ ]:
'''
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8').sample(frac=0.5, random_state=42)
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8').sample(1000)
'''

In [ ]:
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']

In [ ]:
df.categoria.value_counts()

livro        838
maquiagem    788
brinquedo    668
game         622
Name: categoria, dtype: int64

In [ ]:
# função de lematização completa do documento
def lemmatizer_text(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

# função de lematização para os verbos do documento
def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

In [ ]:
!pip install spacy
!python -m spacy download pt

In [ ]:
# teste das funções de lematização
import spacy
nlp = spacy.load('pt')

# validação das funções
print(lemmatizer_text('correndo 1, 2, 3'))
print(lemmatizer_verbs('correndo 1, 2, 3'))

correr 1 , 2 , 3
correr 1 , 2 , 3


In [ ]:
# aplica a lematização no dataframe criando novas colunas
df['text_lemma'] = df.texto.apply(lemmatizer_text)
df['text_lemma_verbs'] = df.texto.apply(lemmatizer_verbs)

In [ ]:
# análise para comparação dos textos
print(df['texto'][0])
print(df['text_lemma'][0])
print(df['text_lemma_verbs'][0])

 O Hobbit - 7ª Ed. 2013  Produto NovoBilbo Bolseiro é um hobbit que leva uma vida confortável e sem ambições. Mas seu contentamento é perturbado quando Gandalf, o mago, e uma companhia de anões batem à sua porta e levam-no para uma expedição. Eles têm um plano para roubar o tesouro guardado por Smaug, o Magnífico, um grande e perigoso dragão. Bilbo reluta muito em participar da aventura, mas acaba surpreendendo até a si mesmo com sua esperteza e sua habilidade como ladrão!CaracterísticasAutor: Tolkien, J. R. R.Peso: 0.44I.S.B.N.: 9788578277109Altura: 20.000000Largura: 13.000000Profundidade: 1.000000Número de Páginas: 328Idioma: PortuguêsAcabamento: BrochuraNúmero da edição: 7Ano da edição: 2013
  O Hobbit - 7ª Ed . 2013   Produto NovoBilbo Bolseiro ser um hobbit que levar umar vidar confortável e sem ambição . Mas seu contentamento ser perturbar quando Gandalf , o mago , e umar companhia de anão bater à suar portar e levam-no parir umar expedição . Eles ter um planar parir roubar o tes

In [ ]:
from sklearn.model_selection import train_test_split

# Divisão das amostras em treino e teste
df_treino, df_teste = train_test_split(
      df, 
      test_size = 0.3, 
      random_state = 42
  )

Árvore de decisão

In [ ]:
# Exemplo 1: vetorização por contagem de termos simples com a combinação de unigrama e bigrama no documento lematizado, sem stopwords do SpaCy e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
import spacy

# stopwords SpaCy
nlp = spacy.load('pt')
stops = nlp.Defaults.stop_words

# vetorização por contagem de termos no documento lematizado
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) # exemplo 1.1: vetorização e combinação de unigrama sem stopwords
#vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) # exemplo 1.2: vetorização e combinação de unigrama e bigrama sem stopwords
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

# treinamento do modelo ávore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_vect_treino, df_treino["categoria"])

# escoragem da classificação na amostra de teste
text_vect_teste = vect.transform(df_teste.text_lemma)
predito = tree.predict(text_vect_teste)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(predito, df_teste["categoria"])

print(text_vect_treino.shape)
print(accuracy)

(2041, 25832)
0.9542857142857143


Exemplo 1.1: 0.9542857142857143

Exemplo 1.2: 0.9417142857142857

In [ ]:
# Exemplo 2: Vetorização por contagem de termos simples com a combinação de unigrama e bigrama no documento com verbos lematizado, sem stopwords do SpaCy e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
import spacy

# stopwords SpaCy
nlp = spacy.load('pt')
stops = nlp.Defaults.stop_words

# vetorização por contagem de termos no documento lematizado
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) # exemplo 2.1: vetorização e combinação de unigrama sem stopwords
#vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) # exemplo 2.2: vetorização e combinação de unigrama e bigrama sem stopwords
vect.fit(df_treino.text_lemma_verbs)
text_vect_treino = vect.transform(df_treino.text_lemma_verbs)

# treinamento do modelo ávore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_vect_treino, df_treino["categoria"])

# escoragem da classificação na amostra de teste
text_vect_teste = vect.transform(df_teste.text_lemma_verbs)
predito = tree.predict(text_vect_teste)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(predito, df_teste["categoria"])

print(text_vect_treino.shape)
print(accuracy)

(2041, 27553)
0.9588571428571429


Exemplo 2.1: 0.9588571428571429

Exemplo 2.2: 0.9542857142857143

In [ ]:
# Exemplo 3: Vetorização por contagem de termos TF-IDF com a combinação de unigrama com documentos lematizado, sem stopwords do SpaCy e NLTK combinadas e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import spacy
import nltk

nltk.download('stopwords')
nlp = spacy.load('pt')

# stopwords do SpaCy e NLTK combinadas
stops_spacy = nlp.Defaults.stop_words
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stops = list(set(stops_spacy).union(set(stops_nltk)))

# vetorização por contagem de termos no documento lematizado
vetorTfidf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops, norm='l2') # exemplo 3.1: vetorização tf-idf e combinação de unigrama sem stopwords NLTK e Spacy
#vetorTfidf = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=stops_spacy, norm='l2') # exemplo 3.2: vetorização tf-idf e combinação de unigrama e bigrama sem stopwords NLTK e Spacy
#vetorTfidf = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=stops_spacy, norm='l1') # exemplo 3.3: vetorização tf e combinação de unigrama e bigrama sem stopwords NLTK e Spacy
vetorTfidf.fit(df_treino.text_lemma)
text_vect_treino = vetorTfidf.transform(df_treino.text_lemma)

# treinamento do modelo ávore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_vect_treino, df_treino["categoria"])

# escoragem da classificação na amostra de teste
text_vect_teste = vetorTfidf.transform(df_teste.text_lemma)
predito = tree.predict(text_vect_teste)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(predito, df_teste["categoria"])

print(text_vect_treino.shape)
print(accuracy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
(2041, 25808)
0.9417142857142857


Exemplo 3.1: 0.9417142857142857

Exemplo 3.2: 0.9417142857142857

Exemplo 3.3: 0.9382857142857143

In [ ]:
# Exemplo 4: Vetorização por contagem de termos simples com a combinação de unigrama e bigrama, documentos com verbos lematizado, sem stopwords do SpaCy e NLTK combinadas e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import spacy
import nltk

nltk.download('stopwords')
nlp = spacy.load('pt')

# stopwords do SpaCy e NLTK combinadas
stops = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))

# vetorização por contagem de termos no documento lematizado
#vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) # vetorização por contagem de termos, combinação de unigrama, remoção das stopwords NLTK e Spacy
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) # vetorização por contagem de termos, combinação de unigrama e bigrama, remoção das stopwords NLTK e Spacy
vect.fit(df_treino.text_lemma_verbs)
text_vect_treino = vect.transform(df_treino.text_lemma_verbs)

# treinamento do modelo ávore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_vect_treino, df_treino["categoria"])

# escoragem da classificação na amostra de teste
text_vect_teste = vect.transform(df_teste.text_lemma_verbs)
predito = tree.predict(text_vect_teste)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(predito, df_teste["categoria"])

print(text_vect_treino.shape)
print(accuracy)

# Nosso melhor modelo até aqui!!! :)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(2041, 131628)
0.96


Exemplo 4.1: 0.9577142857142857

Exemplo 4.2: 0.96

Regressão Logística

In [ ]:
# Exemplo Base: Vetorização por contagem de termos simples com a combinação de unigramas e modelo de Regressão Logística
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# vetorização por contagem de termos no documento lematizado
vect = CountVectorizer()
vect.fit(df_treino.texto)
text_vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(text_vect_treino, df_treino["categoria"])

# escoragem da classificação na amostra de teste
text_vect_teste = vect.transform(df_teste.texto)
predito = model.predict(text_vect_teste)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(predito, df_teste["categoria"])

print(text_vect_treino.shape)
print(accuracy)

(2041, 29957)
0.9805714285714285


In [ ]:
# Exemplo 5: Vetorização por contagem de termos simples com a combinação de unigramas e bigramas, remoção de stopwords NLTK e SpaCy combinadas, texto com verbos lematizados e modelo de Regressão Logística
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import spacy
import nltk

nltk.download('stopwords')
nlp = spacy.load('pt')

# stopwords do SpaCy e NLTK combinadas
stops = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))

# vetorização por contagem de termos no documento lematizado
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) # vetorização por contagem de termos, combinação de unigrama, remoção das stopwords NLTK e Spacy
#vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) # vetorização por contagem de termos, combinação de unigrama e bigrama, remoção das stopwords NLTK e Spacy
vect.fit(df_treino.text_lemma_verbs)
text_vect_treino = vect.transform(df_treino.text_lemma_verbs)

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(text_vect_treino, df_treino["categoria"])

# escoragem da classificação na amostra de teste
text_vect_teste = vect.transform(df_teste.text_lemma_verbs)
predito = model.predict(text_vect_teste)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(predito, df_teste["categoria"])

print(text_vect_treino.shape)
print(accuracy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(2041, 27519)
0.9874285714285714


Exemplo 5.1: 0.9874285714285714

Exemplo 5.2: 0.9874285714285714

In [ ]:
# Exemplo 6: Vetorização por TF-IDF com a combinação de unigramas, remoção de stopwords NLTK e SpaCy combinadas, texto com verbos lematizados e modelo de Regressão Logística
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import spacy
import nltk

nltk.download('stopwords')
nlp = spacy.load('pt')

# stopwords do SpaCy e NLTK combinadas
stops = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))

# vetorização por contagem de termos no documento lematizado
vect = TfidfVectorizer(ngram_range=(1,1), stop_words=stops) # vetorização por contagem de termos, combinação de unigrama, remoção das stopwords NLTK e Spacy
vect.fit(df_treino.text_lemma_verbs)
text_vect_treino = vect.transform(df_treino.text_lemma_verbs)

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(text_vect_treino, df_treino["categoria"])

# escoragem da classificação na amostra de teste
text_vect_teste = vect.transform(df_teste.text_lemma_verbs)
predito = model.predict(text_vect_teste)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(predito, df_teste["categoria"])

print(text_vect_treino.shape)
print(accuracy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(2041, 27519)
0.9851428571428571


In [ ]:
# Exemplo 7: Vetorização por TF-IDF com a combinação de unigramas e bigramas, remoção de stopwords NLTK e SpaCy combinadas, texto lematizado e modelo de Regressão Logística
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import spacy
import nltk

nltk.download('stopwords')
nlp = spacy.load('pt')

# stopwords do SpaCy e NLTK combinadas
stops = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))

# vetorização por contagem de termos no documento lematizado
vect = TfidfVectorizer(ngram_range=(1,1), stop_words=stops) # vetorização por contagem de termos, combinação de unigrama, remoção das stopwords NLTK e Spacy
#vect = TfidfVectorizer(ngram_range=(1,2), stop_words=stops) # vetorização por contagem de termos, combinação de unigrama e bigrama, remoção das stopwords NLTK e Spacy
#vect = TfidfVectorizer(ngram_range=(1,1)) # vetorização por contagem de termos, combinação de unigrama
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(text_vect_treino, df_treino["categoria"])

# escoragem da classificação na amostra de teste
text_vect_teste = vect.transform(df_teste.text_lemma)
predito = model.predict(text_vect_teste)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(predito, df_teste["categoria"])

print(text_vect_treino.shape)
print(accuracy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(2041, 25808)
0.9828571428571429
